# SQL query from table names - Continued

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [8]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [9]:
context = [ {'role':'system', 'content':"""
CREATE TABLE Professors (
    professor_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    department VARCHAR(100) NOT NULL,
    university VARCHAR(100) NOT NULL,
    research_interests TEXT
);
             
CREATE TABLE Reviews (
    review_id INT PRIMARY KEY AUTO_INCREMENT,
    professor_id INT,
    student_name VARCHAR(100) NOT NULL,
    review_date DATE NOT NULL,
    rating INT CHECK (rating BETWEEN 1 AND 5),
    comments TEXT,
    FOREIGN KEY (professor_id) REFERENCES Professors(professor_id)
);
             
CREATE TABLE Institutes (
    institute_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    location VARCHAR(100) NOT NULL,
    type VARCHAR(50) NOT NULL  -- E.g., 'University', 'Research Center', etc.
);             
             
             
             """} ]



In [10]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
INSERT INTO Professors (name, department, university, research_interests)
VALUES 
('Dr. Alice Johnson', 'Computer Science', 'Ohio University', 'Artificial Intelligence, Machine Learning'),
('Dr. John Smith', 'Physics', 'Harvard University', 'Quantum Mechanics, Astrophysics'),
('Dr. Emily Wong', 'Chemistry', 'MIT', 'Organic Chemistry, Nanotechnology');
                 
INSERT INTO Reviews (professor_id, student_name, review_date, rating, comments)
VALUES 
(1, 'Jane Doe', '2023-05-10', 4, 'Great teaching style but difficult exams.'),
(2, 'Mike Brown', '2023-06-20', 5, 'Very insightful lectures and helpful during office hours.'),
(3, 'Sarah White', '2023-07-15', 3, 'Knowledgeable but not very approachable.');
                 
                 INSERT INTO Institutes (name, location, type)
VALUES 
('Ohio University', 'Athens, OH', 'University'),
('Harvard University', 'Cambridge, MA', 'University'),
('MIT Research Center', 'Cambridge, MA', 'Research Center');"""
})

In [11]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [12]:
#new
context_user = context.copy()
print(return_CCRMSQL("""which professors are the most loved by students""", context_user))

To determine which professors are the most loved by students based on the reviews, we can look at the average ratings given by students. Professors with higher average ratings are likely to be more loved by students. Here's the SQL query to find the professors with the highest average ratings:

```sql
SELECT p.name AS Professor_Name, AVG(r.rating) AS Average_Rating
FROM Professors p
JOIN Reviews r ON p.professor_id = r.professor_id
GROUP BY p.professor_id
ORDER BY Average_Rating DESC;
```

This query will provide a list of professors along with their average ratings, sorted in descending order of average rating. Professors with higher average ratings are considered more loved by students.


In [37]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("which professors are the most loved by students", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee who is best paid by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the result by salary in descending order and limits the output to only one row, which corresponds to the employee with the highest salary.


In [38]:
#new
print(return_CCRMSQL("what is the average salary per institution?", context_user))

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [39]:
#old
print(return_CCRMSQL("what is the average salary per institution?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY AVG(sa.salary) DESC
LIMIT 1;
```

This SQL query joins the "studies" and "salary" tables on the ID_usr column. It then calculates the average salary for each institution, orders the results in descending order based on the average salary, and returns the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?